Group Memebers

*   Chirag Gupta - 10623123
*   Chukwuebuka Uwaoma - 10621864
*   Dipanshu



In [12]:
# Import necesarry libraries
import tweepy
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime
import re
import pypyodbc

In [13]:
# Load dotenv and read secret keys and tokens
load_dotenv()
api_key = os.getenv('api_key')
api_secret_key = os.getenv('api_secret_key')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [14]:
# Connect to twitter using tweepy API and stored tokens
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [15]:
# Pulls tweets based on search query and geocode location
tweetlist = []
geocode="53.1424,-7.6921,300km" # "53.3498053,-6.2603097,25km"
filter =  '-filter:retweets AND -filter:replies'
query = 'housing'
tweets = tweepy.Cursor(api.search_tweets, q=query+filter, geocode=geocode, lang='en', tweet_mode="extended", count=20).items(20)

# adds tweets to a dataframe
column_names = ["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]
df = pd.DataFrame(columns=column_names)

for tweet in tweets:
    #create a new row object
    new_row = pd.Series({
        "id": tweet.id, 
        "created_at":tweet.created_at,
        "username": tweet.user.screen_name, 
        "tweet": tweet.full_text, 
        "retweet_count": tweet.retweet_count, 
        "favorite_count": tweet.favorite_count,
        "user_location":tweet.user.location,
    })
    
    # get the index of the last row
    last_row_index = len(df)
    # add the new row to the DataFrame using .iloc[]
    df.loc[last_row_index] = new_row
df.head()
    
# Pull tweet text, id, created_at_date,  username, retweets, likes saved to temporary dataframe - Virgil - DONE

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647749486717059073,2023-04-16 23:51:05+00:00,PressReview99,Bill for housing Ukrainian refugees in Ireland...,0,0,Italia
1,1647737877499977729,2023-04-16 23:04:58+00:00,roisinanna,"It's the tissue thin pancakes, the sad meal pl...",0,2,Dublin
2,1647736084506214402,2023-04-16 22:57:50+00:00,byeyave,Why is the government so bad at managing housi...,0,1,"Co. Louth, ireland"
3,1647724080815120386,2023-04-16 22:10:08+00:00,RoryHearneGaffs,This is the reality of Government created hous...,11,49,Dublin
4,1647710526259036163,2023-04-16 21:16:17+00:00,RoryHearneGaffs,Michael Martin was recently on the @talkingbol...,28,64,Dublin


In [16]:
# Clean the data
    ## clean text 
    ## date_time in right format eg dd/mm/yyyy - Chirag

# connect to a external DB upload cleaned data
    ## make sure tweets are unique by id - Dipanshu

# pull from database assign to dataframe - DP 
    # tokenize - virgil
    # lematize  - Dipanshu
    # visualise word cloud - Chirag

In [17]:
# Pre-processing the data

# Formatting the date to dd/mm/YYYY format
def formatDate(timestamp):
    # As the timestamp retrieved from the api is already in DatetimeIndex form, simply running string format function 
    # of the datetime library and passing the desired format
    formatted_date = timestamp.strftime("%d/%m/%Y")
    return formatted_date

# Getting only the words and numbers from the tweet that contains links, special symbols, emojis, etc.
def cleanTweet(tweet):
    # Converting the string to lowercase and replacing the ' with blank
    removed_apostrophe = tweet.lower().replace("'","")
    # Using regex pattern to replace the links, <html> tags and &amp; symbol (seen frequently in the tweets) with blank
    removed_links_tags_frequentsymbol = re.sub("(https?://\S+)|(<[\w]+>)|(&amp;)","",removed_apostrophe)
    # This regex pattern returns a list that contains all the words and numbers in the tweet
    cleaned_tweet = re.findall("([a-z0-9]+)", removed_links_tags_frequentsymbol)
    cleaned_tweet = " ".join(cleaned_tweet)
    return cleaned_tweet

# Applying respective functions to both columns in the dataset
df["created_at"] = df["created_at"].apply(lambda x: formatDate(x))
df["tweet"] = df["tweet"].apply(lambda x: cleanTweet(x))
df.head()

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647749486717059073,16/04/2023,PressReview99,bill for housing ukrainian refugees in ireland...,0,0,Italia
1,1647737877499977729,16/04/2023,roisinanna,its the tissue thin pancakes the sad meal plan...,0,2,Dublin
2,1647736084506214402,16/04/2023,byeyave,why is the government so bad at managing housi...,0,1,"Co. Louth, ireland"
3,1647724080815120386,16/04/2023,RoryHearneGaffs,this is the reality of government created hous...,11,49,Dublin
4,1647710526259036163,16/04/2023,RoryHearneGaffs,michael martin was recently on the talkingboll...,28,64,Dublin


In [18]:
connection = mysql.connector.connect(host='localhost',
                                         database='ca',
                                         user='root',
                                         password='toor')

In [19]:
mySql_insert_query = 'CREATE TABLE if not exists api (id varchar(255) primary key ,created_at varchar(255),username varchar(255),tweet TEXT(50000),retweet_count varchar(255),favorite_count varchar(255),user_location varchar(255));'

In [20]:
cursor = connection.cursor(buffered=True)
cursor.execute(mySql_insert_query)
connection.commit()

In [21]:
for i in range(df.shape[0]):
    s=""
    for j in df.columns:
        s=s+j+","
    s=s[:-1]

    t=""
    for k in df.columns:
        t=t+"'"+str(df[k].iloc[i])+"',"
    t=t[:-1]
    command="INSERT INTO api ("+s+") VALUES ("+t+");"
    print(command)
    try:
        cursor.execute(command)
    except:
        pass
connection.commit()

INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647749486717059073','16/04/2023','PressReview99','bill for housing ukrainian refugees in ireland to hit 1 billion within weeks youtube food dukeandduchessofsussex news princeharry','0','0','Italia');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647737877499977729','16/04/2023','roisinanna','its the tissue thin pancakes the sad meal planning and the total destruction of our social housing structure not necessarily in that order','0','2','Dublin ');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647736084506214402','16/04/2023','byeyave','why is the government so bad at managing housing i mean the country is having a housing crisis thats so bad its caused a second diaspora yet if you look around any town youll notice the amount of derelict buildings that could easily be fixed up and 

In [41]:
# Data retrieve from DB and insert into pd df

sql_query = "SELECT * FROM api"
cursor.execute(sql_query)
result = cursor.fetchall()
result
data=pd.DataFrame(result)
data.columns=["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]
data


,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647634340082335750,16/04/2023,UrbanArchCon,6 collective housing interventions that rethin...,0,0,"South Dublin, Ireland"
1,1647635355229724672,16/04/2023,luke_oriordan,minister for housing must explain department u...,0,2,"Glasnevin, Dublin"
2,1647641119440744456,16/04/2023,RoryHearneGaffs,irelands housing crisis generation catastrophe...,35,117,Dublin
3,1647655780730929152,16/04/2023,oceanclub,quite a gulf between this anon fine gael tds s...,2,8,"Dublin, Ireland"
4,1647656641074982916,16/04/2023,john_e_horan,really enjoyed this and feels true ireland now...,0,0,"South Dublin, Ireland"
5,1647660755959918592,16/04/2023,picturesby_JOE,kiosk clontarf promenade dublin corporation ho...,1,4,"Dublin City, Ireland"
6,1647676862787715074,16/04/2023,JoeSeward1,im starting to think the three toughest jobs o...,0,0,"Cork City, Ireland"
7,1647678790468812802,16/04/2023,MaryCallaghanSD,100k vacant homes in ireland in the middle of ...,0,0,"Dublin City, Ireland"
8,1647683852213669891,16/04/2023,kevintoconnor,fine gael is a very chilling party its tds are...,15,33,Dublin
9,1647692554261999623,16/04/2023,winkenswexford,tract housing copy paste houses housingcrisis ...,0,0,"Wexford, Ireland"
